<a href="https://colab.research.google.com/github/MariaMuu/Experimenting/blob/main/Rag_RecursiveCharacter_chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-core chromadb langchain-text-splitters

In [133]:
import os
from google.colab import userdata

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://eu.api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = "maria first"

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [144]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai.embeddings import OpenAIEmbeddings

loader = WebBaseLoader('https://sumsub.com/blog/crypto-regulations-in-the-european-union-markets-in-crypto-assets-mica/')
doc = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

chunks = text_splitter.split_documents(doc)
print(f"Total number of chunks: {len(chunks)}")

for x,chunk in enumerate(chunks[:10]):
  print(f"document {x+1}:\n{chunk.page_content}\n---\n")

Total number of chunks: 67
document 1:
MiCA Regulation and EU Crypto Rules: What Changes in 2026Skip to contentExpert-led compliance mediaJan 13, 202611 min readMiCA Regulation and EU Crypto Rules: What Changes in 2026MiCA has reshaped EU crypto regulation. Learn what‚Äôs changed, when to be ready, and what EU crypto firms must do to stay compliant.Luke Owain BoultContent WriterAnastasia SakharovaHead of Fintech Compliance at SumsubThe EU’s Markets in Crypto Assets (MiCA) regulation is a huge step for crypto regulations in the EU and beyond. On December 30, 2024, MiCA fully came into effect, and the transitional grandfathering
---

document 2:
into effect, and the transitional grandfathering period began. This phase, which is soon coming to an end, allows existing crypto-asset service providers (CASPs) that were operating legally before this date to continue their services in the EU without immediate compliance with the new regulations, provided they apply for authorization within the 

In [145]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = OpenAIEmbeddings()
)

In [151]:
#retriever = vectorstore.as_retriever(search_kwargs = {'k':3})

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,
        "fetch_k": 15,
        "lambda_mult": 0.5
    }
)

In [152]:
relevant_documents = retriever.invoke("What's mica?")

#print(relevant_documents[0].page_content)

for i, doc in enumerate(relevant_documents):
    print(f"Document {i+1}:\n{doc.page_content}\n---\n")

Document 1:
MiCA covers crypto-assets not already regulated under existing EU financial laws, including asset-referenced tokens (ARTs), e-money tokens (EMTs), and crypto assets other than ARTs and EMTs (e.g., utility tokens). It does not generally apply to traditional financial instruments, deposits, insurance products, or most non-fungible tokens (NFTs).
            



            Who needs a MiCA license?
---

Document 2:
MiCA also applies to CASPs, which are legal entities or other undertakings that professionally provide one or more crypto-asset services to clients as their main occupation or business.
Crypto-asset services include:
Providing custody and administration of crypto-assets on behalf of clients
Operating a trading platform for crypto-assets
Exchanging crypto-assets for funds
Exchanging crypto-assets for other crypto-assets
Executing orders for crypto-assets on behalf of clients
Placing of crypto-assets
Reception and transmission of orders for crypto-assets on behalf of